<a href="https://colab.research.google.com/github/Mariano1985-design/Data-Science-I---84650/blob/main/PrecioPropiedades_Foletto_84650.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstract
El dataset seleccionado es un listado de propiedades Mexicana por tipo con la cantidad y tipo de habitaciones, metros cuadros, amenities y ubicación, con el precio de cada una.</font>

Este análisis tiene como objetivo entender la relación del precio y las variables descriptas en el dataset, poder tener un valor de referencia que permita poder estimar precios por tipo de propiedad.</font>

Elegi este dataset para luego poder aplicar esto en casos prácticos en mi trabajo, por ejemplo entender como relacionar multivariables y poder identificar como cada una impacta en el resultado.</font>

#  Preguntas e Hipótesis
Con las variables disponibles se encontrara un patrón para poder predecir el precio de una casa.

La principal variable que influye en el precio de la propiedad son los m2.

Entender si la antigüedad de la casa influye en un menor precio de la propiedad y cuánto es la relación.

# Análisis del dataset

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [53]:
# Importación de archivo csv que esta guardado en Github
url = 'https://raw.githubusercontent.com/Mariano1985-design/Data-Science-I---84650/main/kc_house_data.csv'
df_casas = pd.read_csv(url)
print(df_casas.head())

           id             date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900.0         3       1.00         1180   
1  6414100192  20141209T000000  538000.0         3       2.25         2570   
2  5631500400  20150225T000000  180000.0         2       1.00          770   
3  2487200875  20141209T000000  604000.0         4       3.00         1960   
4  1954400510  20150218T000000  510000.0         3       2.00         1680   

   sqft_lot  floors  waterfront  view  ...  grade  sqft_above  sqft_basement  \
0      5650     1.0           0     0  ...      7        1180              0   
1      7242     2.0           0     0  ...      7        2170            400   
2     10000     1.0           0     0  ...      6         770              0   
3      5000     1.0           0     0  ...      7        1050            910   
4      8080     1.0           0     0  ...      8        1680              0   

   yr_built  yr_renovated  zipcode      lat     lo

In [54]:
# Información general del DataFrame (tipos de datos, nulos, etc.)
df_casas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [55]:
# Ver las columnas disponibles
print(f"\nColumnas: {df_casas.columns.tolist()}")


Columnas: ['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


In [56]:
# Cantidad de filas y columnas del dataframe
print(f"\nDimensiones del DataFrame: {df_casas.shape}")


Dimensiones del DataFrame: (21613, 21)


In [57]:
df_casas['date'] = pd.to_datetime(df_casas['date'])
df_casas.dtypes['date']

dtype('<M8[ns]')

In [58]:
df_casas.describe()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,21613,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,...,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,2014-10-29 04:38:01.959931648,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,...,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
min,1.000102e+06,2014-05-02 00:00:00,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,...,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,2014-07-22 00:00:00,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,...,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,2014-10-16 00:00:00,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,...,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,2015-02-17 00:00:00,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,...,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,2015-05-27 00:00:00,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,...,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000
std,2.876566e+09,NaN,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,...,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631


In [59]:
promedio_precio = df_casas['price'].mean()
mediana_precio = df_casas['price'].median()
moda_precio = df_casas['price'].mode()
desviacion_estandar_precio = df_casas['price'].std()

print(f"El promedio del precio es: {promedio_precio}")
print(f"La mediana del precio es: {mediana_precio}")
print(f"La moda del precio es:\n{moda_precio}")
print(f"La desviación estándar del precio es: {desviacion_estandar_precio}")

El promedio del precio es: 540088.1417665294
La mediana del precio es: 450000.0
La moda del precio es:
0    350000.0
1    450000.0
Name: price, dtype: float64
La desviación estándar del precio es: 367127.1964826997
